**Step 1: Setup TensorFlow 1.x**

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import tensorflow as tf
print(tf.__version__)

1.15.2


**Step 2: Clone the tensorflow-yolo-v3 repository**

In [ ]:
# Clone
!git clone https://github.com/mystic123/tensorflow-yolo-v3

Cloning into 'tensorflow-yolo-v3'...
remote: Enumerating objects: 226, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 226 (delta 2), reused 2 (delta 0), pack-reused 219
Receiving objects: 100% (226/226), 63.05 KiB | 1.58 MiB/s, done.
Resolving deltas: 100% (114/114), done.


**Step 3a: Mount Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 AIMLProject3	    test.csv		     'Website flow.png'
'Colab Notebooks'   train.csv		      YOLOv3_TFLite
 Prediction.ipynb  'Untitled document.gdoc'


**Step 3b: Navigate to tensorflow-yolo-v3 repository**

In [ ]:
%cd tensorflow-yolo-v3/

/content/tensorflow-yolo-v3


**Step 4: Download .weights and .classes files**

In [ ]:
!gdown --id 1AzD7m1gXN1e_RGwxdKH2kQnykuKCzThy  #classes.txt
!gdown --id 1pqMn_pH-b4VwpO0_HYFImPY4nKKd2fX0  #yolov3_training_last.weights

Downloading...
From: https://drive.google.com/uc?id=1AzD7m1gXN1e_RGwxdKH2kQnykuKCzThy
To: /content/tensorflow-yolo-v3/classes.txt
100% 72.0/72.0 [00:00<00:00, 115kB/s]
Downloading...
From: https://drive.google.com/uc?id=1pqMn_pH-b4VwpO0_HYFImPY4nKKd2fX0
To: /content/tensorflow-yolo-v3/yolov3-tiny-bdd100k_last.weights
100% 34.8M/34.8M [00:01<00:00, 22.9MB/s]


**Step 5: Convert YOLOv3 .weights to .pb**

In [ ]:
#!python convert_weights_pb.py --class_names "classes.txt" --weights_file "yolov3-tiny-bdd100k_last.weights" --data_format "NHWC" 

In [ ]:
!python3 convert_weights_pb.py --class_names "classes.txt" --data_format NHWC --weights_file "yolov3-tiny-bdd100k_last.weights" --tiny

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0130 04:18:14.601224 140717583611776 module_wrapper.py:139] From convert_weights_pb.py:42: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.


W0130 04:18:14.603292 140717583611776 module_wrapper.py:139] From convert_weights_pb.py:44: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

Instructions for updating:
Please use `layer.__call__` method instead.
W0130 04:18:14.606392 140717583611776 deprecation.py:323] From /tensorflow-1.15.2/python3.7/tensorflow_core/contrib/layers/python/layers/layers.py:1057: Layer.apply (from tensorflow.python.keras.engine.

**Step 6a: Restart runtime**

Click on Runtime on the menu bar and select **Restart runtime.**

**Step 6b: Setup TensorFlow 2.x**

In [ ]:
%tensorflow_version 2.x

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.7.0


In [ ]:
import tempfile
import os
import glob

**Step 7a: Navigate to tensorflow-yolo-v3 repository**

In [ ]:
%cd tensorflow-yolo-v3/

/content/tensorflow-yolo-v3


**Step 7b: Check input and output nodes of neural network**


In [ ]:
import tensorflow.compat.v1 as tf

In [ ]:
gf = tf.GraphDef()   
m_file = open('frozen_darknet_yolov3_model.pb','rb')
gf.ParseFromString(m_file.read())

with open('somefile.txt', 'a') as the_file:
     for n in gf.node:
         the_file.write(n.name+'\n')

file = open('somefile.txt','r')
data = file.readlines()
output = data[len(data)-1]
print("Output array = ", output)

file.seek ( 0 )
input=file.readline()
print("Input array = ", input)

Output array =  output_boxes

Input array =  inputs



**Step 8: Convert frozen_darknet_yolov3_model.pb to .tflite**

In [ ]:
# Reference: https://github.com/sayakpaul/Adventures-in-TensorFlow-Lite/blob/master/DeepLabV3/DeepLab_TFLite_COCO.ipynb
# Load the TensorFlow model
# The preprocessing and the post-processing steps should not be included in the TF Lite model graph 
# because some operations (ArgMax) might not support the delegates. 
# Insepct the graph using Netron https://lutzroeder.github.io/netron/
converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(
    graph_def_file='/content/tensorflow-yolo-v3/frozen_darknet_yolov3_model.pb', 
    input_arrays = ['inputs'],   # Here, 'inputs' is the value of input array from Step 7b
    output_arrays = ['output_boxes'], # Here, 'output_boxes' is the value of output array from Step 7b
    input_shapes={'inputs': [1, 416, 416, 3]} # Here, 'inputs' is the value of input array from Step 7b
)

# Optional: Perform the simplest optimization known as post-training dynamic range quantization.
# https://www.tensorflow.org/lite/performance/post_training_quantization#dynamic_range_quantization
# You can refer to the same document for other types of optimizations.
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Convert to TFLite Model
tflite_model = converter.convert()

_, dynamic_tflite_path = tempfile.mkstemp('.tflite')
tflite_model_size = open(dynamic_tflite_path, 'wb').write(tflite_model)
tf_model_size = os.path.getsize('/content/tensorflow-yolo-v3/frozen_darknet_yolov3_model.pb')
print('TensorFlow Model is  {} bytes'.format(tf_model_size))
print('TFLite Model is      {} bytes'.format(tflite_model_size))
print('Post training dynamic range quantization saves {} bytes'.format(tf_model_size-tflite_model_size))

TensorFlow Model is  34821859 bytes
TFLite Model is      8820320 bytes
Post training dynamic range quantization saves 26001539 bytes


In [ ]:
!ls -lh {dynamic_tflite_path}

-rw------- 1 root root 8.5M Jan 30 04:20 /tmp/tmpeytp10pn.tflite


**Step 9: Move the .tflite model into your Google Drive folder**

In [ ]:
tflite_file = ''
tflite_file = tflite_file.join(glob.glob("/tmp/*.tflite")) # Find the temp file of type .tflite
tflite_file = tflite_file[5:len(tflite_file)]  # Extract .tflite file name.

%cd ..
%cd ../tmp
os.rename(tflite_file, 'detect.tflite') # Rename the temp .tflite to detect.tflite
%mv detect.tflite ../content/gdrive/MyDrive/YOLOv3_TFLite/  # Move detect.tflite to Goolge Drive

/content
/tmp
